In [1]:
library(tidyverse)
library(mediation)
library(brms)
library(dplyr)
library(tidyr)
library(lme4)
library(ggplot2)

── Attaching core tidyverse packages ─────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   4.0.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.1.0     
── Conflicts ───────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors


Warning messages:
1: package ‘tidyverse’ was built under R version 4.3.3 
2: package ‘lubridate’ was built under R version 4.3.2 


Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: mvtnorm
Loading required package: sandwich
mediation: Causal Mediation Analysis
Version: 4.5.1



Warning messages:
1: package ‘mvtnorm’ was built under R version 4.3.3 
2: package ‘sandwich’ was built under R version 4.3.3 


Loading required package: Rcpp
Loading 'brms' package (version 2.22.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').

Attaching package: ‘brms’

The following object is masked from ‘package:stats’:

    ar



Warning message:
package ‘brms’ was built under R version 4.3.3 



Attaching package: ‘lme4’

The following object is masked from ‘package:brms’:

    ngrps



# Import Data

In [17]:
# --------------------------------------------------
# LOAD DATA
# --------------------------------------------------
df_long <- read.csv("df_RCT.csv")

# --------------------------------------------------
# VARIABLES
# --------------------------------------------------

outcomes <- c("fss_sum", "woods_sum", "dsq_sum", "dsq_freq_sum", "dsq_severity_sum", "psqi_sum", "psqi_disturbances")

resp_vars <- c(
  "mip_pre_max", "smip_pre_max", "fit_pre_max", "id_pre_max", "slopesmip_pre_max",
  "sindex_pre_max", "pif_pre_max", "volume_pre_max",
  "mip_post_max", "smip_post_max", "fit_post_max", "id_post_max", "slopesmip_post_max",
  "pif_post_max", "sindex_post_max", "volume_post_max",
  "cpet_vo2peak_absolute", "cpet_ve", "cpet_vt_peak", "cpet_o2pulse",
  "fmd_percent"
)

covariates <- c("data_age", "data_centimeters", "data_kilograms", "subject_female")

# --------------------------------------------------
# STEP 1 — CENTER RESP VARIABLES & CREATE NEW COLUMNS
# --------------------------------------------------

for (v in resp_vars) {
  if (!v %in% names(df_long)) {
    warning("Variable ", v, " not found in df_long, skipping.")
  } else {
    mean_v <- mean(df_long[[v]], na.rm = TRUE)
    new_name <- paste0(v, "_c")
    df_long[[new_name]] <- df_long[[v]] - mean_v
  }
}

# --------------------------------------------------
# STEP 2 — CREATE df_prepost (BEST STRUCTURE)
# 2 rows per participant: pre-treadmill (0) and post-treadmill (1)
# USING ONLY time_point = 0 VISIT
# --------------------------------------------------

# Helper function to rename pre/post columns to unified names
rename_resp <- function(df, prefix) {
  rename_at(
    df,
    vars(ends_with("_pre_max")),
    ~gsub("_pre_max", paste0("_", prefix), .)
  ) %>%
  rename_at(
    vars(ends_with("_post_max")),
    ~gsub("_post_max", paste0("_", prefix), .)
  )
}

# Keep ONLY time_point 0 (pre-intervention visit)
df_baseline <- df_long %>% filter(time_point == 1)

# PRE-treadmill dataset (condition = 0)
df_pre <- df_baseline %>%
  mutate(condition = 0) %>%
  dplyr::select(
    record_id, condition, all_of(outcomes), all_of(covariates),
    matches("_pre_max$"),  # original values
    matches("_pre_max_c$") # centered values
  ) %>%
  rename_with(~gsub("_pre_max", "", .x), matches("_pre_max$")) %>%
  rename_with(~gsub("_pre_max_c", "_c", .x), matches("_pre_max_c$"))

# POST-treadmill dataset (condition = 1)
df_post <- df_baseline %>%
  mutate(condition = 1) %>%
  dplyr::select(
    record_id, condition, all_of(outcomes), all_of(covariates),
    matches("_post_max$"),
    matches("_post_max_c$")
  ) %>%
  rename_with(~gsub("_post_max", "", .x), matches("_post_max$")) %>%
  rename_with(~gsub("_post_max_c", "_c", .x), matches("_post_max_c$"))

# Combine into final df_prepost
df_prepost <- bind_rows(df_pre, df_post) %>%
  arrange(record_id, condition)


# --------------------------------------------------
# Export
# --------------------------------------------------
write.csv(df_prepost, file = "df_prepost.csv", row.names = FALSE)


# Normality Checks

In [11]:
library(dplyr)

# Variables to check
outcomes <- c("fss_sum", "dsq_sum")
resp_core <- c("mip", "sindex", "pif")

vars_to_check <- c(
  outcomes,
  paste0(resp_core, "_pre_max"),
  paste0(resp_core, "_post_max")
)

# Use baseline visit rows
df_norm <- df_long %>%
  filter(time_point == 1)

shapiro_list <- list()
idx <- 1

pdf("normality_diagnostics.pdf", width = 8, height = 6)

for (v in vars_to_check) {
  if (!v %in% names(df_norm)) next
  
  x <- df_norm[[v]]
  x <- x[!is.na(x)]
  if (length(x) < 5) next
  
  # Shapiro-Wilk
  sh <- shapiro.test(x)
  shapiro_list[[idx]] <- data.frame(
    variable = v,
    n        = length(x),
    W        = unname(sh$statistic),
    p_value  = sh$p.value,
    stringsAsFactors = FALSE
  )
  idx <- idx + 1
  
  # Plots: histogram + density, and Q-Q
  par(mfrow = c(1, 2))
  
  hist(
    x, breaks = "FD", freq = FALSE,
    main = paste("Histogram & Density:", v),
    xlab = v
  )
  lines(density(x), lwd = 2)
  
  qqnorm(x, main = paste("Q-Q Plot:", v))
  qqline(x, col = 2)
}

dev.off()

normality_shapiro_results <- bind_rows(shapiro_list)
write.csv(
  normality_shapiro_results,
  "normality_shapiro_results.csv",
  row.names = FALSE
)

cat("✅ Normality diagnostics saved:\n")
cat("   • normality_diagnostics.pdf\n")
cat("   • normality_shapiro_results.csv\n")


✅ Normality diagnostics saved:
   • normality_diagnostics.pdf
   • normality_shapiro_results.csv


# Bivariate Correlations

In [ ]:
# Function to calculate both Pearson and Spearman correlations
calculate_correlations <- function(data, outcome, respiratory, data_source) {
  # Remove rows with missing values for these two variables
  clean_data <- data[complete.cases(data[c(outcome, respiratory)]), ]
  
  # Calculate Pearson correlation
  pearson_result <- cor.test(clean_data[[outcome]], clean_data[[respiratory]], 
                             method = "pearson")
  
  # Calculate Spearman correlation
  spearman_result <- cor.test(clean_data[[outcome]], clean_data[[respiratory]], 
                              method = "spearman")
  
  # Return as a data frame
  tibble(
    data_source = data_source,
    outcome = outcome,
    respiratory = respiratory,
    pearson_r = pearson_result$estimate,
    pearson_p = pearson_result$p.value,
    spearman_rho = spearman_result$estimate,
    spearman_p = spearman_result$p.value,
    n = nrow(clean_data)
  )
}

# Initialize results list
correlation_results <- list()

# Calculate correlations for all combinations for BOTH datasets
i <- 1
for (y in outcomes) {
  for (x in resp_vars) {
    # Combined Visit 1 + Visit 2 data
    correlation_results[[i]] <- calculate_correlations(df_long, y, x, "combined_visits")
    i <- i + 1
    
    # Visit 1 only data
    visit1_data <- df_long %>% filter(time_point == 1)
    correlation_results[[i]] <- calculate_correlations(visit1_data, y, x, "visit1_only")
    i <- i + 1
  }
}

# Combine all results
final_correlations <- bind_rows(correlation_results)

# Reorder columns to match your requested format
final_correlations <- final_correlations %>%
  dplyr::select(data_source, outcome, respiratory, pearson_r, pearson_p, spearman_rho, spearman_p, n)

# Print a sample to check
head(final_correlations)

# Save to CSV
write.csv(final_correlations, "bivariate_correlations_all_data.csv", row.names = FALSE)

# Create formatted summary tables sorted by strongest Spearman correlation
summary_correlations <- final_correlations %>%
  group_by(data_source, outcome) %>%
  arrange(data_source, outcome, desc(abs(spearman_rho))) %>%
  mutate(
    pearson_sig = case_when(
      pearson_p < 0.001 ~ "***",
      pearson_p < 0.01 ~ "**",
      pearson_p < 0.05 ~ "*",
      TRUE ~ ""
    ),
    spearman_sig = case_when(
      spearman_p < 0.001 ~ "***",
      spearman_p < 0.01 ~ "**",
      spearman_p < 0.05 ~ "*",
      TRUE ~ ""
    )
  )

# Print summary tables for easy viewing
cat("COMBINED VISITS (1+2) - Correlations with FSS (Fatigue):\n")
summary_correlations %>% 
  filter(data_source == "combined_visits" & outcome == "fss_sum") %>% 
  print(n = 20)

cat("\nCOMBINED VISITS (1+2) - Correlations with Woods MFI (Brain Fog):\n")
summary_correlations %>% 
  filter(data_source == "combined_visits" & outcome == "woods_sum") %>% 
  print(n = 20)

cat("\nVISIT 1 ONLY - Correlations with FSS (Fatigue):\n")
summary_correlations %>% 
  filter(data_source == "visit1_only" & outcome == "fss_sum") %>% 
  print(n = 20)

cat("\nVISIT 1 ONLY - Correlations with Woods MFI (Brain Fog):\n")
summary_correlations %>% 
  filter(data_source == "visit1_only" & outcome == "woods_sum") %>% 
  print(n = 20)

# Save the sorted summary
write.csv(summary_correlations, "sorted_correlations_summary_all_data.csv", row.names = FALSE)

# Optional: Create a wide format for easy comparison between datasets
wide_format <- final_correlations %>%
  dplyr::select(-pearson_p, -spearman_p) %>%
  pivot_wider(
    names_from = data_source,
    values_from = c(pearson_r, spearman_rho, n),
    names_glue = "{data_source}_{.value}"
  ) %>%
  dplyr::select(outcome, respiratory, 
         visit1_only_pearson_r, combined_visits_pearson_r,
         visit1_only_spearman_rho, combined_visits_spearman_rho,
         visit1_only_n, combined_visits_n)

# Save wide format for easy comparison
write.csv(wide_format, "correlations_wide_format_comparison.csv", row.names = FALSE)

cat("\nWide Format Comparison (first 10 rows):\n")
print(head(wide_format, 10))

# Steiger Correlation Test

## 111pm One sided p-value + Spearman Rank

In [18]:
# --------------------------------------------------
# LIBRARIES
# --------------------------------------------------
library(dplyr)
library(psych)   # for r.test

# --------------------------------------------------
# 1. CONFIGURATION
# --------------------------------------------------

# Outcomes (fatigue / PEM / symptom scales)
outcomes <- c("fss_sum", "woods_sum", "dsq_sum", "dsq_freq_sum", "dsq_severity_sum", "psqi_sum", "psqi_disturbances")

# Respiratory pre/post PAIRS (raw + percent-predicted)
# Edit this list as needed.
resp_pairs <- list(
  # raw max values
  list(label = "mip",    pre = "mip_pre_max",    post = "mip_post_max"),
  list(label = "smip",   pre = "smip_pre_max",   post = "smip_post_max"),
  list(label = "fit",    pre = "fit_pre_max",    post = "fit_post_max"),
  list(label = "id",     pre = "id_pre_max",     post = "id_post_max"),
  list(label = "slopesmip", pre = "slopesmip_pre_max", post = "slopesmip_post_max"),
  list(label = "sindex", pre = "sindex_pre_max", post = "sindex_post_max"),
  list(label = "pif",    pre = "pif_pre_max",    post = "pif_post_max"),
  list(label = "volume", pre = "volume_pre_max", post = "volume_post_max"),
  
  # percent-predicted variants
  list(label = "mip_pp_1", pre = "mip_pre_max_percentpredict_1",
                          post = "mip_post_max_percentpredict_1"),
  list(label = "mip_pp_2", pre = "mip_pre_max_percentpredict_2",
                          post = "mip_post_max_percentpredict_2"),
  list(label = "mip_pp_3", pre = "mip_pre_max_percentpredict_3",
                          post = "mip_post_max_percentpredict_3"),
  list(label = "mip_pp_4", pre = "mip_pre_max_percentpredict_4",
                          post = "mip_post_max_percentpredict_4"),
  list(label = "mip_pp_5", pre = "mip_pre_max_percentpredict_5",
                          post = "mip_post_max_percentpredict_5"),
  list(label = "sindex_pp", pre = "sindex_pre_max_percentpredict",
                            post = "sindex_post_max_percentpredict")
)

# Number of bootstrap resamples for CI on (r_post - r_pre)
B <- 2000

# --------------------------------------------------
# 2. DEFINE BASELINE DATASET (ONE ROW PER PARTICIPANT)
# --------------------------------------------------
# If you've already defined df_baseline earlier, you can skip this.
df_baseline <- df_long %>%
  filter(time_point == 1)

# --------------------------------------------------
# 3. HELPER: Fisher CI for a correlation
# --------------------------------------------------
fisher_ci <- function(r, n, alpha = 0.05) {
  if (is.na(r) || abs(r) >= 1 || n <= 3) return(c(NA_real_, NA_real_))
  z  <- atanh(r)
  se <- 1 / sqrt(n - 3)
  z_crit <- qnorm(1 - alpha / 2)
  lo <- z - z_crit * se
  hi <- z + z_crit * se
  c(tanh(lo), tanh(hi))
}

# --------------------------------------------------
# 4. MAIN LOOP: SPEARMAN + ONE-SIDED STEIGER
#     H1: |r_post| > |r_pre|  (post-treadmill association is stronger)
# --------------------------------------------------
steiger_results <- list()
res_idx <- 1

for (y in outcomes) {
  if (!y %in% names(df_baseline)) {
    warning("Outcome ", y, " not found in df_baseline, skipping.")
    next
  }
  
  for (pair in resp_pairs) {
    
    resp_label <- pair$label
    pre_var    <- pair$pre
    post_var   <- pair$post
    
    # Check that both variables exist
    if (!all(c(pre_var, post_var) %in% names(df_baseline))) {
      warning("Skipping ", resp_label, " for outcome ", y,
              ": ", pre_var, " or ", post_var, " not found.")
      next
    }
    
    # Build analysis dataset
    df_sub <- df_baseline %>%
      dplyr::select(all_of(c(y, pre_var, post_var))) %>%
      na.omit()
    
    n <- nrow(df_sub)
    if (n < 10) {
      warning("Skipping ", resp_label, " for outcome ", y,
              ": too few complete cases (n = ", n, ").")
      next
    }
    
    y_vals <- df_sub[[y]]
    x_pre  <- df_sub[[pre_var]]
    x_post <- df_sub[[post_var]]
    
    # Spearman correlations
    r_pre      <- suppressWarnings(cor(y_vals, x_pre,  method = "spearman"))
    r_post     <- suppressWarnings(cor(y_vals, x_post, method = "spearman"))
    r_xprepost <- suppressWarnings(cor(x_pre,  x_post, method = "spearman"))
    
    if (any(is.na(c(r_pre, r_post, r_xprepost)))) {
      warning("Skipping ", resp_label, " for outcome ", y,
              ": one or more correlations are NA.")
      next
    }
    
    # Steiger test using Spearman r's
    steiger_obj <- psych::r.test(
      n   = n,
      r12 = r_pre,
      r13 = r_post,
      r23 = r_xprepost
    )
    
    t_val <- steiger_obj$t
    p_two <- steiger_obj$p
    
    # Difference in correlation (post - pre)
    r_diff <- r_post - r_pre
    
    # Direction for one-sided H1: |r_post| > |r_pre|
    # If effect is in hypothesized direction, p_one = p_two / 2
    # Otherwise, p_one = 1 - p_two / 2 (conservative)
    if (abs(r_post) > abs(r_pre)) {
      direction <- "post_stronger"
      p_one     <- p_two / 2
    } else {
      direction <- "pre_stronger_or_equal"
      p_one     <- 1 - p_two / 2
    }
    
    # Bootstrap CI for r_diff (Spearman-based)
    set.seed(123)  # for reproducibility
    boot_diffs <- numeric(B)
    
    for (b in seq_len(B)) {
      idx_b    <- sample.int(n, size = n, replace = TRUE)
      y_b      <- y_vals[idx_b]
      x_pre_b  <- x_pre[idx_b]
      x_post_b <- x_post[idx_b]
      
      r_pre_b  <- suppressWarnings(cor(y_b, x_pre_b,  method = "spearman"))
      r_post_b <- suppressWarnings(cor(y_b, x_post_b, method = "spearman"))
      
      if (is.na(r_pre_b) || is.na(r_post_b)) {
        boot_diffs[b] <- NA_real_
      } else {
        boot_diffs[b] <- r_post_b - r_pre_b
      }
    }
    
    boot_diffs <- boot_diffs[!is.na(boot_diffs)]
    if (length(boot_diffs) > 10) {
      ci_lower <- quantile(boot_diffs, probs = 0.025)
      ci_upper <- quantile(boot_diffs, probs = 0.975)
    } else {
      ci_lower <- ci_upper <- NA_real_
    }
    
    # Fisher-type CI for r_pre and r_post (approximate, using Spearman)
    ci_pre  <- fisher_ci(r_pre,  n)
    ci_post <- fisher_ci(r_post, n)
    
    steiger_results[[res_idx]] <- data.frame(
      outcome         = y,
      resp_type       = resp_label,
      n               = n,
      r_pre           = r_pre,
      r_pre_ci_low    = ci_pre[1],
      r_pre_ci_high   = ci_pre[2],
      r_post          = r_post,
      r_post_ci_low   = ci_post[1],
      r_post_ci_high  = ci_post[2],
      r_diff          = r_diff,
      r_diff_ci_low   = ci_lower,
      r_diff_ci_high  = ci_upper,
      r_xprepost      = r_xprepost,
      t_steiger       = t_val,
      p_steiger_two   = p_two,
      direction_H1    = direction,
      p_steiger_one   = p_one,   # one-sided for H1: |r_post| > |r_pre|
      stringsAsFactors = FALSE
    )
    
    res_idx <- res_idx + 1
  }
}

# --------------------------------------------------
# 5. COMBINE AND SAVE RESULTS
# --------------------------------------------------
steiger_spearman_one_sided <- dplyr::bind_rows(steiger_results)

write.csv(
  steiger_spearman_one_sided,
  "steiger_spearman_one_sided_results.csv",
  row.names = FALSE
)

cat("\n✅ DONE!\n")
cat("• Spearman Steiger tests (one-sided H1: |r_post| > |r_pre|) completed\n")
cat("• Results saved as: steiger_spearman_one_sided_results.csv\n")



✅ DONE!
• Spearman Steiger tests (one-sided H1: |r_post| > |r_pre|) completed
• Results saved as: steiger_spearman_one_sided_results.csv


## 1252 pm

In [9]:
# ==================================================
# Steiger dependent correlation tests:
#   r(Y, X_pre) vs r(Y, X_post) for same participants
# ==================================================

library(dplyr)
library(tidyr)
library(ggplot2)

# We need the 'psych' package for Steiger's r.test
if (!requireNamespace("psych", quietly = TRUE)) {
  stop("Please install the 'psych' package: install.packages('psych')")
}
library(psych)

# --------------------------------------------------
# 1. CONFIGURATION
# --------------------------------------------------

# Outcomes (fatigue / PEM / symptom scales)
outcomes <- c("fss_sum", "woods_sum", "dsq_sum")

# Respiratory variables that have *_pre_max and *_post_max
resp_types <- c("mip", "smip", "fit", "id", "slopesmip", "sindex", "pif", "volume")

# Number of bootstrap resamples for CI on (r_post - r_pre)
B <- 2000

# --------------------------------------------------
# 2. BASELINE DATA ONLY (time_point = 0)
# --------------------------------------------------
# Assumes df_long is already loaded from df_RCT.csv

df_baseline <- df_long %>%
  filter(time_point == 1)

# Quick sanity check
cat("Number of baseline rows:", nrow(df_baseline), "\n")

# --------------------------------------------------
# 3. STORAGE FOR RESULTS
# --------------------------------------------------

steiger_results <- list()
res_idx <- 1

# --------------------------------------------------
# 4. MAIN LOOP: outcomes × resp_types
# --------------------------------------------------

for (y in outcomes) {
  if (!y %in% names(df_baseline)) {
    warning("Outcome ", y, " not found in df_baseline, skipping.")
    next
  }
  
  for (resp in resp_types) {
    
    pre_var  <- paste0(resp, "_pre_max")
    post_var <- paste0(resp, "_post_max")
    
    # Check that both variables exist
    if (!all(c(pre_var, post_var) %in% names(df_baseline))) {
      warning("Skipping ", resp, " for outcome ", y,
              ": ", pre_var, " or ", post_var, " not found.")
      next
    }
    
    # Build analysis dataset
    df_sub <- df_baseline %>%
      dplyr::select(all_of(c(y, pre_var, post_var))) %>%
      na.omit()
    
    n <- nrow(df_sub)
    if (n < 10) {
      warning("Skipping ", resp, " for outcome ", y,
              ": too few complete cases (n = ", n, ").")
      next
    }
    
    # -----------------------
    # 4a. Compute correlations
    # -----------------------
    y_vals    <- df_sub[[y]]
    x_pre     <- df_sub[[pre_var]]
    x_post    <- df_sub[[post_var]]
    
    r_pre       <- suppressWarnings(cor(y_vals, x_pre,   use = "complete.obs"))
    r_post      <- suppressWarnings(cor(y_vals, x_post,  use = "complete.obs"))
    r_xprepost  <- suppressWarnings(cor(x_pre,  x_post,  use = "complete.obs"))
    
    # Check for NA correlations (e.g., zero variance)
    if (any(is.na(c(r_pre, r_post, r_xprepost)))) {
      warning("Skipping ", resp, " for outcome ", y,
              ": one or more correlations are NA (likely zero variance).")
      next
    }
    
    # -----------------------
    # 4b. Steiger's dependent correlation test
    # -----------------------
    # psych::r.test for overlapping correlations:
    # both correlations share Y; X_pre and X_post are correlated.
    steiger_obj <- psych::r.test(
      n  = n,
      r12 = r_pre,       # r(Y, X_pre)
      r13 = r_post,      # r(Y, X_post)
      r23 = r_xprepost   # r(X_pre, X_post)
    )
    
    # r.test returns a list with fields: t, p, etc.
    t_val <- steiger_obj$t
    p_val <- steiger_obj$p
    
    # Difference in correlations
    r_diff <- r_post - r_pre
    
    # -----------------------
    # 4c. Bootstrap CI for (r_post - r_pre)
    # -----------------------
    set.seed(123)  # for reproducibility; adjust if you like
    
    boot_diffs <- numeric(B)
    
    for (b in seq_len(B)) {
      idx_b <- sample.int(n, size = n, replace = TRUE)
      y_b      <- y_vals[idx_b]
      x_pre_b  <- x_pre[idx_b]
      x_post_b <- x_post[idx_b]
      
      r_pre_b      <- suppressWarnings(cor(y_b, x_pre_b,  use = "complete.obs"))
      r_post_b     <- suppressWarnings(cor(y_b, x_post_b, use = "complete.obs"))
      
      # If something degenerates (e.g., zero variance in a bootstrap sample), skip that draw
      if (is.na(r_pre_b) || is.na(r_post_b)) {
        boot_diffs[b] <- NA_real_
      } else {
        boot_diffs[b] <- r_post_b - r_pre_b
      }
    }
    
    boot_diffs <- boot_diffs[!is.na(boot_diffs)]
    if (length(boot_diffs) > 10) {
      ci_lower <- quantile(boot_diffs, probs = 0.025)
      ci_upper <- quantile(boot_diffs, probs = 0.975)
    } else {
      ci_lower <- NA_real_
      ci_upper <- NA_real_
    }
    
    # -----------------------
    # 4d. Optional: Fisher z CI for r_pre and r_post individually
    # -----------------------
    fisher_ci <- function(r, n, alpha = 0.05) {
      if (abs(r) >= 1 || n <= 3) return(c(NA_real_, NA_real_))
      z  <- atanh(r)
      se <- 1 / sqrt(n - 3)
      z_crit <- qnorm(1 - alpha/2)
      lo <- z - z_crit * se
      hi <- z + z_crit * se
      c(tanh(lo), tanh(hi))
    }
    
    ci_pre  <- fisher_ci(r_pre,  n)
    ci_post <- fisher_ci(r_post, n)
    
    # -----------------------
    # 4e. Store results
    # -----------------------
    steiger_results[[res_idx]] <- data.frame(
      outcome       = y,
      resp_type     = resp,
      n             = n,
      r_pre         = r_pre,
      r_pre_ci_low  = ci_pre[1],
      r_pre_ci_high = ci_pre[2],
      r_post        = r_post,
      r_post_ci_low  = ci_post[1],
      r_post_ci_high = ci_post[2],
      r_diff        = r_diff,
      r_diff_ci_low = ci_lower,
      r_diff_ci_high= ci_upper,
      r_xprepost    = r_xprepost,
      t_steiger     = t_val,
      p_steiger     = p_val,
      stringsAsFactors = FALSE
    )
    
    res_idx <- res_idx + 1
  }
}

# --------------------------------------------------
# 5. COMBINE & SAVE RESULTS
# --------------------------------------------------

steiger_df <- bind_rows(steiger_results)

write.csv(
  steiger_df,
  "steiger_dependent_correlation_results.csv",
  row.names = FALSE
)

cat("\n✅ Steiger tests complete.\n")
cat("Results saved to: steiger_dependent_correlation_results.csv\n")


# --------------------------------------------------
# 6. SCATTERPLOTS: Y vs X_pre and Y vs X_post
#    for visual comparison (fixed version)
# --------------------------------------------------

pdf("steiger_scatterplots.pdf", width = 8, height = 6)

if (nrow(steiger_df) > 0) {
  for (i in seq_len(nrow(steiger_df))) {
    y        <- steiger_df$outcome[i]
    resp     <- steiger_df$resp_type[i]
    pre_var  <- paste0(resp, "_pre_max")
    post_var <- paste0(resp, "_post_max")
    
    if (!all(c(y, pre_var, post_var) %in% names(df_baseline))) next
    
    # Build a clean plotting frame explicitly
    df_plot <- df_baseline %>%
      transmute(
        outcome = .data[[y]],
        pre  = .data[[pre_var]],
        post = .data[[post_var]]
      ) %>%
      tidyr::pivot_longer(
        cols = c(pre, post),
        names_to  = "state",
        values_to = "resp_value"
      ) %>%
      na.omit()
    
    p_scatter <- ggplot(
      df_plot,
      aes(x = resp_value, y = outcome, color = state)
    ) +
      geom_point() +
      geom_smooth(method = "lm", se = FALSE) +
      labs(
        title = paste0(
          "Outcome: ", y, " | Respiratory: ", resp,
          "\nPre vs Post treadmill association"
        ),
        x = paste0(resp, " (pre/post)"),
        y = y,
        color = "State"
      )
    
    print(p_scatter)
  }
} else {
  plot.new()
  title("No valid outcome × respiratory combinations found.")
}

dev.off()

cat("Scatterplots saved to: steiger_scatterplots.pdf\n")


# ==================================================
# ADD PERCENT-PREDICTED RESP VARIABLES
# ==================================================

# Define pairs of pre/post percent-predicted variables
resp_pp_pairs <- list(
  list(label = "mip_pp_1",
       pre   = "mip_pre_max_percentpredict_1",
       post  = "mip_post_max_percentpredict_1"),
  list(label = "mip_pp_2",
       pre   = "mip_pre_max_percentpredict_2",
       post  = "mip_post_max_percentpredict_2"),
  list(label = "mip_pp_3",
       pre   = "mip_pre_max_percentpredict_3",
       post  = "mip_post_max_percentpredict_3"),
  list(label = "mip_pp_4",
       pre   = "mip_pre_max_percentpredict_4",
       post  = "mip_post_max_percentpredict_4"),
  list(label = "mip_pp_5",
       pre   = "mip_pre_max_percentpredict_5",
       post  = "mip_post_max_percentpredict_5"),
  list(label = "sindex_pp",
       pre   = "sindex_pre_max_percentpredict",
       post  = "sindex_post_max_percentpredict")
)

pp_results <- list()
pp_idx <- 1

for (y in outcomes) {
  if (!y %in% names(df_baseline)) next
  
  for (pair in resp_pp_pairs) {
    resp_label <- pair$label
    pre_var    <- pair$pre
    post_var   <- pair$post
    
    # Check variables exist
    if (!all(c(pre_var, post_var) %in% names(df_baseline))) {
      warning("Skipping ", resp_label, " for outcome ", y,
              ": ", pre_var, " or ", post_var, " not found.")
      next
    }
    
    # Build analysis dataset
    df_sub <- df_baseline %>%
      dplyr::select(all_of(c(y, pre_var, post_var))) %>%
      na.omit()
    
    n <- nrow(df_sub)
    if (n < 10) {
      warning("Skipping ", resp_label, " for outcome ", y,
              ": too few complete cases (n = ", n, ").")
      next
    }
    
    y_vals   <- df_sub[[y]]
    x_pre    <- df_sub[[pre_var]]
    x_post   <- df_sub[[post_var]]
    
    r_pre      <- suppressWarnings(cor(y_vals, x_pre,  use = "complete.obs"))
    r_post     <- suppressWarnings(cor(y_vals, x_post, use = "complete.obs"))
    r_xprepost <- suppressWarnings(cor(x_pre,  x_post, use = "complete.obs"))
    
    if (any(is.na(c(r_pre, r_post, r_xprepost)))) next
    
    steiger_obj <- psych::r.test(
      n   = n,
      r12 = r_pre,
      r13 = r_post,
      r23 = r_xprepost
    )
    
    t_val  <- steiger_obj$t
    p_val  <- steiger_obj$p
    r_diff <- r_post - r_pre
    
    # Bootstrap CI for r_diff
    set.seed(123)
    B <- 2000
    boot_diffs <- numeric(B)
    
    for (b in seq_len(B)) {
      idx_b  <- sample.int(n, size = n, replace = TRUE)
      y_b    <- y_vals[idx_b]
      x_pre_b  <- x_pre[idx_b]
      x_post_b <- x_post[idx_b]
      
      r_pre_b  <- suppressWarnings(cor(y_b, x_pre_b,  use = "complete.obs"))
      r_post_b <- suppressWarnings(cor(y_b, x_post_b, use = "complete.obs"))
      
      if (is.na(r_pre_b) || is.na(r_post_b)) {
        boot_diffs[b] <- NA_real_
      } else {
        boot_diffs[b] <- r_post_b - r_pre_b
      }
    }
    
    boot_diffs <- boot_diffs[!is.na(boot_diffs)]
    if (length(boot_diffs) > 10) {
      ci_lower <- quantile(boot_diffs, probs = 0.025)
      ci_upper <- quantile(boot_diffs, probs = 0.975)
    } else {
      ci_lower <- ci_upper <- NA_real_
    }
    
    # Fisher CIs for r_pre, r_post as before
    fisher_ci <- function(r, n, alpha = 0.05) {
      if (abs(r) >= 1 || n <= 3) return(c(NA_real_, NA_real_))
      z  <- atanh(r)
      se <- 1 / sqrt(n - 3)
      z_crit <- qnorm(1 - alpha/2)
      lo <- z - z_crit * se
      hi <- z + z_crit * se
      c(tanh(lo), tanh(hi))
    }
    
    ci_pre  <- fisher_ci(r_pre,  n)
    ci_post <- fisher_ci(r_post, n)
    
    pp_results[[pp_idx]] <- data.frame(
      outcome       = y,
      resp_type     = resp_label,
      n             = n,
      r_pre         = r_pre,
      r_pre_ci_low  = ci_pre[1],
      r_pre_ci_high = ci_pre[2],
      r_post        = r_post,
      r_post_ci_low  = ci_post[1],
      r_post_ci_high = ci_post[2],
      r_diff        = r_diff,
      r_diff_ci_low = ci_lower,
      r_diff_ci_high= ci_upper,
      r_xprepost    = r_xprepost,
      t_steiger     = t_val,
      p_steiger     = p_val,
      stringsAsFactors = FALSE
    )
    
    pp_idx <- pp_idx + 1
  }
}

pp_df <- dplyr::bind_rows(pp_results)

# Combine with original Steiger results and overwrite the CSV
steiger_df_all <- dplyr::bind_rows(steiger_df, pp_df)

write.csv(
  steiger_df_all,
  "steiger_dependent_correlation_results_with_pp.csv",
  row.names = FALSE
)

cat("\n✅ Percent-predicted Steiger tests added.\n")
cat("Combined results saved to: steiger_dependent_correlation_results_with_pp.csv\n")

Number of baseline rows: 22 

✅ Steiger tests complete.
Results saved to: steiger_dependent_correlation_results.csv
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geo

## OG

In [ ]:
# ==================================================
# Steiger dependent correlation tests:
#   r(Y, X_pre) vs r(Y, X_post) for same participants
# ==================================================

library(dplyr)
library(tidyr)
library(ggplot2)

# We need the 'psych' package for Steiger's r.test
if (!requireNamespace("psych", quietly = TRUE)) {
  stop("Please install the 'psych' package: install.packages('psych')")
}
library(psych)

# --------------------------------------------------
# 1. CONFIGURATION
# --------------------------------------------------

# Outcomes (fatigue / PEM / symptom scales)
outcomes <- c("fss_sum", "woods_sum", "dsq_sum")

# Respiratory variables that have *_pre_max and *_post_max
resp_types <- c("mip", "smip", "fit", "id", "slopesmip", "sindex", "pif", "volume")

# Number of bootstrap resamples for CI on (r_post - r_pre)
B <- 2000

# --------------------------------------------------
# 2. BASELINE DATA ONLY (time_point = 0)
# --------------------------------------------------
# Assumes df_long is already loaded from df_RCT.csv

df_baseline <- df_long %>%
  filter(time_point == 1)

# Quick sanity check
cat("Number of baseline rows:", nrow(df_baseline), "\n")

# --------------------------------------------------
# 3. STORAGE FOR RESULTS
# --------------------------------------------------

steiger_results <- list()
res_idx <- 1

# --------------------------------------------------
# 4. MAIN LOOP: outcomes × resp_types
# --------------------------------------------------

for (y in outcomes) {
  if (!y %in% names(df_baseline)) {
    warning("Outcome ", y, " not found in df_baseline, skipping.")
    next
  }
  
  for (resp in resp_types) {
    
    pre_var  <- paste0(resp, "_pre_max")
    post_var <- paste0(resp, "_post_max")
    
    # Check that both variables exist
    if (!all(c(pre_var, post_var) %in% names(df_baseline))) {
      warning("Skipping ", resp, " for outcome ", y,
              ": ", pre_var, " or ", post_var, " not found.")
      next
    }
    
    # Build analysis dataset
    df_sub <- df_baseline %>%
      dplyr::select(all_of(c(y, pre_var, post_var))) %>%
      na.omit()
    
    n <- nrow(df_sub)
    if (n < 10) {
      warning("Skipping ", resp, " for outcome ", y,
              ": too few complete cases (n = ", n, ").")
      next
    }
    
    # -----------------------
    # 4a. Compute correlations
    # -----------------------
    y_vals    <- df_sub[[y]]
    x_pre     <- df_sub[[pre_var]]
    x_post    <- df_sub[[post_var]]
    
    r_pre       <- suppressWarnings(cor(y_vals, x_pre,   use = "complete.obs"))
    r_post      <- suppressWarnings(cor(y_vals, x_post,  use = "complete.obs"))
    r_xprepost  <- suppressWarnings(cor(x_pre,  x_post,  use = "complete.obs"))
    
    # Check for NA correlations (e.g., zero variance)
    if (any(is.na(c(r_pre, r_post, r_xprepost)))) {
      warning("Skipping ", resp, " for outcome ", y,
              ": one or more correlations are NA (likely zero variance).")
      next
    }
    
    # -----------------------
    # 4b. Steiger's dependent correlation test
    # -----------------------
    # psych::r.test for overlapping correlations:
    # both correlations share Y; X_pre and X_post are correlated.
    steiger_obj <- psych::r.test(
      n  = n,
      r12 = r_pre,       # r(Y, X_pre)
      r13 = r_post,      # r(Y, X_post)
      r23 = r_xprepost   # r(X_pre, X_post)
    )
    
    # r.test returns a list with fields: t, p, etc.
    t_val <- steiger_obj$t
    p_val <- steiger_obj$p
    
    # Difference in correlations
    r_diff <- r_post - r_pre
    
    # -----------------------
    # 4c. Bootstrap CI for (r_post - r_pre)
    # -----------------------
    set.seed(123)  # for reproducibility; adjust if you like
    
    boot_diffs <- numeric(B)
    
    for (b in seq_len(B)) {
      idx_b <- sample.int(n, size = n, replace = TRUE)
      y_b      <- y_vals[idx_b]
      x_pre_b  <- x_pre[idx_b]
      x_post_b <- x_post[idx_b]
      
      r_pre_b      <- suppressWarnings(cor(y_b, x_pre_b,  use = "complete.obs"))
      r_post_b     <- suppressWarnings(cor(y_b, x_post_b, use = "complete.obs"))
      
      # If something degenerates (e.g., zero variance in a bootstrap sample), skip that draw
      if (is.na(r_pre_b) || is.na(r_post_b)) {
        boot_diffs[b] <- NA_real_
      } else {
        boot_diffs[b] <- r_post_b - r_pre_b
      }
    }
    
    boot_diffs <- boot_diffs[!is.na(boot_diffs)]
    if (length(boot_diffs) > 10) {
      ci_lower <- quantile(boot_diffs, probs = 0.025)
      ci_upper <- quantile(boot_diffs, probs = 0.975)
    } else {
      ci_lower <- NA_real_
      ci_upper <- NA_real_
    }
    
    # -----------------------
    # 4d. Optional: Fisher z CI for r_pre and r_post individually
    # -----------------------
    fisher_ci <- function(r, n, alpha = 0.05) {
      if (abs(r) >= 1 || n <= 3) return(c(NA_real_, NA_real_))
      z  <- atanh(r)
      se <- 1 / sqrt(n - 3)
      z_crit <- qnorm(1 - alpha/2)
      lo <- z - z_crit * se
      hi <- z + z_crit * se
      c(tanh(lo), tanh(hi))
    }
    
    ci_pre  <- fisher_ci(r_pre,  n)
    ci_post <- fisher_ci(r_post, n)
    
    # -----------------------
    # 4e. Store results
    # -----------------------
    steiger_results[[res_idx]] <- data.frame(
      outcome       = y,
      resp_type     = resp,
      n             = n,
      r_pre         = r_pre,
      r_pre_ci_low  = ci_pre[1],
      r_pre_ci_high = ci_pre[2],
      r_post        = r_post,
      r_post_ci_low  = ci_post[1],
      r_post_ci_high = ci_post[2],
      r_diff        = r_diff,
      r_diff_ci_low = ci_lower,
      r_diff_ci_high= ci_upper,
      r_xprepost    = r_xprepost,
      t_steiger     = t_val,
      p_steiger     = p_val,
      stringsAsFactors = FALSE
    )
    
    res_idx <- res_idx + 1
  }
}

# --------------------------------------------------
# 5. COMBINE & SAVE RESULTS
# --------------------------------------------------

steiger_df <- bind_rows(steiger_results)

write.csv(
  steiger_df,
  "steiger_dependent_correlation_results.csv",
  row.names = FALSE
)

cat("\n✅ Steiger tests complete.\n")
cat("Results saved to: steiger_dependent_correlation_results.csv\n")


# --------------------------------------------------
# 6. SCATTERPLOTS: Y vs X_pre and Y vs X_post
#    for visual comparison (fixed version)
# --------------------------------------------------

pdf("steiger_scatterplots.pdf", width = 8, height = 6)

if (nrow(steiger_df) > 0) {
  for (i in seq_len(nrow(steiger_df))) {
    y        <- steiger_df$outcome[i]
    resp     <- steiger_df$resp_type[i]
    pre_var  <- paste0(resp, "_pre_max")
    post_var <- paste0(resp, "_post_max")
    
    if (!all(c(y, pre_var, post_var) %in% names(df_baseline))) next
    
    # Build a clean plotting frame explicitly
    df_plot <- df_baseline %>%
      transmute(
        outcome = .data[[y]],
        pre  = .data[[pre_var]],
        post = .data[[post_var]]
      ) %>%
      tidyr::pivot_longer(
        cols = c(pre, post),
        names_to  = "state",
        values_to = "resp_value"
      ) %>%
      na.omit()
    
    p_scatter <- ggplot(
      df_plot,
      aes(x = resp_value, y = outcome, color = state)
    ) +
      geom_point() +
      geom_smooth(method = "lm", se = FALSE) +
      labs(
        title = paste0(
          "Outcome: ", y, " | Respiratory: ", resp,
          "\nPre vs Post treadmill association"
        ),
        x = paste0(resp, " (pre/post)"),
        y = y,
        color = "State"
      )
    
    print(p_scatter)
  }
} else {
  plot.new()
  title("No valid outcome × respiratory combinations found.")
}

dev.off()

cat("Scatterplots saved to: steiger_scatterplots.pdf\n")


Number of baseline rows: 22 

✅ Steiger tests complete.
Results saved to: steiger_dependent_correlation_results.csv


: Error in `pivot_longer()`:
! Can't rename variables in this context.

# SUR

## 459pm

In [46]:
library(systemfit)
library(car)
library(dplyr)
library(purrr)
library(broom)

# ------------------------------------------------------------
# 1. Filter to baseline only (no post-intervention data)
# ------------------------------------------------------------
df_t1 <- df_long %>% filter(time_point == 1)

# ------------------------------------------------------------
# 2. Define outcomes, respiratory pairs, and COVARIATES
# ------------------------------------------------------------
outcomes <- c("fss_sum", "woods_sum", "dsq_sum")

resp_pairs <- list(
  mip = c("mip_pre_max", "mip_post_max"),
  smip = c("smip_pre_max", "smip_post_max"),
  fit = c("fit_pre_max", "fit_post_max"),
  id = c("id_pre_max", "id_post_max"),
  slopesmip = c("slopesmip_pre_max", "slopesmip_post_max"),
  sindex = c("sindex_pre_max", "sindex_post_max"),
  pif = c("pif_pre_max", "pif_post_max"),
  volume = c("volume_pre_max", "volume_post_max")
)

# COVARIATES
covars <- c("subject_female", "data_age", "data_centimeters", "data_kilograms")
# covars <- NULL  # Uncomment for unadjusted

# ------------------------------------------------------------
# 3. DETAILED DIAGNOSTIC for first model
# ------------------------------------------------------------
cat("\n=== DETAILED DIAGNOSTIC ===\n")
out <- outcomes[1]
resp <- names(resp_pairs)[1]
pre_var <- resp_pairs[[resp]][1]
post_var <- resp_pairs[[resp]][2]

if (!is.null(covars) && length(covars) > 0) {
  needed_vars <- c(out, pre_var, post_var, covars)
} else {
  needed_vars <- c(out, pre_var, post_var)
}

d_test <- df_t1 %>%
  dplyr::select(all_of(needed_vars)) %>%
  na.omit()

cat("\nTest data dimensions:", nrow(d_test), "rows ×", ncol(d_test), "cols\n")
cat("Column names:", paste(names(d_test), collapse=", "), "\n")
cat("\nChecking for any constant columns:\n")
for(col in names(d_test)) {
  if(length(unique(d_test[[col]])) == 1) {
    cat("  WARNING:", col, "is constant!\n")
  }
}

# Try building formulas
if (!is.null(covars) && length(covars) > 0) {
  covar_string <- paste(covars, collapse = " + ")
  rhs_pre <- paste(pre_var, covar_string, sep = " + ")
  rhs_post <- paste(post_var, covar_string, sep = " + ")
} else {
  rhs_pre <- pre_var
  rhs_post <- post_var
}

f_pre <- as.formula(paste(out, "~", rhs_pre))
f_post <- as.formula(paste(out, "~", rhs_post))

cat("\nFormula 1:", deparse(f_pre), "\n")
cat("Formula 2:", deparse(f_post), "\n")

# Try fitting individual OLS models first
cat("\n--- Testing individual OLS models ---\n")
ols1 <- tryCatch(
  lm(f_pre, data = d_test),
  error = function(e) {
    cat("OLS1 failed:", e$message, "\n")
    return(NULL)
  }
)
if(!is.null(ols1)) cat("OLS1 succeeded\n")

ols2 <- tryCatch(
  lm(f_post, data = d_test),
  error = function(e) {
    cat("OLS2 failed:", e$message, "\n")
    return(NULL)
  }
)
if(!is.null(ols2)) cat("OLS2 succeeded\n")

# Now try systemfit with more verbose error catching
cat("\n--- Testing systemfit ---\n")
sur_test <- tryCatch({
  cat("Creating equation list...\n")
  eqns <- list(pre = f_pre, post = f_post)
  cat("Equation list created\n")
  cat("Calling systemfit...\n")
  result <- systemfit(
    eqns,
    method = "SUR",
    data = d_test,
    control = list(solvetol = 1e-10)
  )
  cat("systemfit succeeded!\n")
  result
}, error = function(e) {
  cat("systemfit FAILED\n")
  cat("Error message:", e$message, "\n")
  cat("Error call:", deparse(e$call), "\n")
  traceback()
  return(NULL)
})

# ------------------------------------------------------------
# 4. MAIN LOOP (only if diagnostic passes)
# ------------------------------------------------------------
if (!is.null(sur_test)) {
  cat("\n=== DIAGNOSTIC PASSED - Running full analysis ===\n\n")
  
  sur_results <- list()
  
  for (out in outcomes) {
    for (resp in names(resp_pairs)) {
      
      pre_var <- resp_pairs[[resp]][1]
      post_var <- resp_pairs[[resp]][2]
      
      if (!is.null(covars) && length(covars) > 0) {
        needed_vars <- c(out, pre_var, post_var, covars)
      } else {
        needed_vars <- c(out, pre_var, post_var)
      }
      
      if (!all(needed_vars %in% names(df_t1))) {
        message("Skipping ", out, " × ", resp, " (missing variables)")
        next
      }
      
      d <- df_t1 %>%
        dplyr::select(all_of(needed_vars)) %>%
        na.omit()
      
      if (nrow(d) < 12) {
        message("Skipping ", out, " × ", resp, " (n < 12)")
        next
      }
      
      cor_pre_post <- cor(d[[pre_var]], d[[post_var]])
      if (abs(cor_pre_post) > 0.95) {
        message("WARNING: ", out, " × ", resp, 
                " has very high correlation (r=", round(cor_pre_post, 3), ")")
      }
      
      # Build formulas
      if (!is.null(covars) && length(covars) > 0) {
        covar_string <- paste(covars, collapse = " + ")
        rhs_pre <- paste(pre_var, covar_string, sep = " + ")
        rhs_post <- paste(post_var, covar_string, sep = " + ")
      } else {
        rhs_pre <- pre_var
        rhs_post <- post_var
      }
      
      f_pre <- as.formula(paste(out, "~", rhs_pre))
      f_post <- as.formula(paste(out, "~", rhs_post))
      
      # Fit SUR
      fits <- tryCatch(
        systemfit(
          list(pre = f_pre, post = f_post),
          method = "SUR",
          data = d,
          control = list(solvetol = 1e-10)
        ),
        error = function(e) {
          message("SUR failed for ", out, " × ", resp, ": ", e$message)
          return(NULL)
        }
      )
      
      if (is.null(fits)) next
      
      sum_fit <- tryCatch(
        summary(fits),
        error = function(e) {
          message("Summary failed for ", out, " × ", resp, ": ", e$message)
          return(NULL)
        }
      )
      
      if (is.null(sum_fit)) next
      
      coef_tab <- sum_fit$coefficients
      p_col <- grep("^Pr", colnames(coef_tab), value = TRUE)[1]
      
      if (is.na(p_col)) {
        message("No p-value column found for ", out, " × ", resp)
        next
      }
      
      rn_pre <- paste0("pre_", pre_var)
      rn_post <- paste0("post_", post_var)
      
      if (!(rn_pre %in% rownames(coef_tab)) || !(rn_post %in% rownames(coef_tab))) {
        message("Missing beta rows for ", out, " × ", resp)
        next
      }
      
      beta_pre <- coef_tab[rn_pre, "Estimate"]
      beta_post <- coef_tab[rn_post, "Estimate"]
      p_pre <- coef_tab[rn_pre, p_col]
      p_post <- coef_tab[rn_post, p_col]
      
      # Wald test
      lh <- tryCatch(
        car::linearHypothesis(
          fits,
          paste0(rn_pre, " - ", rn_post, " = 0")
        ),
        error = function(e) {
          message("linearHypothesis failed for ", out, " × ", resp, ": ", e$message)
          return(NULL)
        }
      )
      
      if (is.null(lh)) next
      
      test_stat_col <- NULL
      test_stat_val <- NA
      if ("Chisq" %in% colnames(lh)) {
        test_stat_col <- "Chisq"
        test_stat_val <- lh[2, "Chisq"]
      } else if ("F" %in% colnames(lh)) {
        test_stat_col <- "F"
        test_stat_val <- lh[2, "F"]
      }
      
      p_test_col <- grep("^Pr\\(>", colnames(lh), value = TRUE)[1]
      
      if (is.na(test_stat_col) || is.na(p_test_col)) {
        message("Could not find test statistic or p-value for ", out, " × ", resp)
        next
      }
      
      p_two <- lh[2, p_test_col]
      
      if (!is.finite(test_stat_val) || !is.finite(p_two)) {
        message("Invalid test statistic for ", out, " × ", resp)
        next
      }
      
      p_one <- p_two / 2
      stronger_side <- ifelse(beta_post > beta_pre, "post", "pre")
      
      sur_results[[length(sur_results) + 1]] <- data.frame(
        outcome = out,
        resp_type = resp,
        N = nrow(d),
        beta_post = beta_post,
        p_post = p_post,
        beta_pre = beta_pre,
        p_pre = p_pre,
        beta_diff = beta_post - beta_pre,
        test_stat = test_stat_val,
        test_type = test_stat_col,
        p_two_tailed = p_two,
        p_one_tailed = p_one,
        stronger_side = stronger_side,
        cor_pre_post = cor(d[[pre_var]], d[[post_var]]),
        stringsAsFactors = FALSE
      )
    }
  }
  
  sur_df <- bind_rows(sur_results)
  write.csv(sur_df, "SUR_beta_comparison_results_with_pvals.csv", row.names = FALSE)
  cat("\nSaved: SUR_beta_comparison_results_with_pvals.csv\n")
  print(sur_df)
  
} else {
  cat("\n=== DIAGNOSTIC FAILED - Cannot proceed ===\n")
  cat("Please check the detailed diagnostic output above\n")
}


=== DETAILED DIAGNOSTIC ===

Test data dimensions: 22 rows × 7 cols
Column names: fss_sum, mip_pre_max, mip_post_max, subject_female, data_age, data_centimeters, data_kilograms 

Checking for any constant columns:

Formula 1: fss_sum ~ mip_pre_max + subject_female + data_age + data_centimeters +      data_kilograms 
Formula 2: fss_sum ~ mip_post_max + subject_female + data_age + data_centimeters +      data_kilograms 

--- Testing individual OLS models ---
OLS1 succeeded
OLS2 succeeded

--- Testing systemfit ---
Creating equation list...
Equation list created
Calling systemfit...
systemfit FAILED
Error message: argument is of length zero 
Error call: if (control$useMatrix) {     xMatEqAttr <- list()     for (i in 1:nEq) {         xMatEqAttr[[i]] <- attributes(xMatEq[[i]])         xMatEq[[i]] <- as(xMatEq[[i]], "denseMatrix")         if (!is.null(inst)) {             zMatEq[[i]] <- as(zMatEq[[i]], "denseMatrix")         }     } } 
9: stop(gettextf("'a' is computationally singular, rcon

## 447pm

In [45]:
library(systemfit)
library(car)
library(dplyr)
library(purrr)
library(broom)

# ------------------------------------------------------------
# 1. Filter to baseline only (no post-intervention data)
# ------------------------------------------------------------
df_t1 <- df_long %>%
  filter(time_point == 1)

# ------------------------------------------------------------
# 2. Define outcomes, respiratory pairs, and COVARIATES
#    >>> EDIT THIS VECTOR TO CUSTOMIZE COVARIATES <<<
# ------------------------------------------------------------
outcomes <- c("fss_sum", "woods_sum", "dsq_sum")

resp_pairs <- list(
  mip        = c("mip_pre_max",       "mip_post_max"),
  smip       = c("smip_pre_max",      "smip_post_max"),
  fit        = c("fit_pre_max",       "fit_post_max"),
  id         = c("id_pre_max",        "id_post_max"),
  slopesmip  = c("slopesmip_pre_max", "slopesmip_post_max"),
  sindex     = c("sindex_pre_max",    "sindex_post_max"),
  pif        = c("pif_pre_max",       "pif_post_max"),
  volume     = c("volume_pre_max",    "volume_post_max")
)

# <<< You can change this to whatever set of covariates you want >>>
covars <- c("subject_female", "data_age", "data_centimeters", "data_kilograms")
# Example: to run UNADJUSTED SUR, set: covars <- character(0)

# ------------------------------------------------------------
# 3. LOOP: Run SUR for each outcome × respiratory predictor
# ------------------------------------------------------------
sur_results <- list()

for (out in outcomes) {
  for (resp in names(resp_pairs)) {
    
    pre_var  <- resp_pairs[[resp]][1]
    post_var <- resp_pairs[[resp]][2]
    
    needed_vars <- c(out, pre_var, post_var, covars)
    
    # Skip if any required columns are missing
    if (!all(needed_vars %in% names(df_t1))) {
      message("Skipping ", out, " × ", resp, " (missing variables)")
      next
    }
    
    # Subset and drop rows with missing values
    d <- df_t1 %>%
      dplyr::select(all_of(needed_vars)) %>%
      na.omit()
    
    # Avoid unstable fits
    if (nrow(d) < 12) {
      message("Skipping ", out, " × ", resp, " (n < 12)")
      next
    }
    
    # --------------------------
    # Build formulas
    # --------------------------
    if (length(covars) > 0) {
      rhs_pre  <- paste(pre_var,  paste(covars, collapse = " + "), sep = " + ")
      rhs_post <- paste(post_var, paste(covars, collapse = " + "), sep = " + ")
    } else {
      rhs_pre  <- pre_var
      rhs_post <- post_var
    }
    
    f_pre  <- as.formula(paste(out, "~", rhs_pre))
    f_post <- as.formula(paste(out, "~", rhs_post))
    
    # --------------------------
    # Fit SUR models
    # --------------------------
    fits <- tryCatch(
      systemfit(
        list(pre = f_pre, post = f_post),
        method = "SUR",
        data = d
      ),
      error = function(e) {
        message("SUR failed for ", out, " × ", resp, ": ", e$message)
        return(NULL)
      }
    )
    
    if (is.null(fits)) next
    
    # --------------------------
    # Extract coefficient table
    # --------------------------
    sum_fit <- summary(fits)
    coef_tab <- sum_fit$coefficients
    
    # Identify p-value column name (e.g., "Pr(>|t|)" or similar)
    p_col <- grep("^Pr", colnames(coef_tab), value = TRUE)[1]
    
    if (is.na(p_col)) {
      message("No p-value column found for ", out, " × ", resp, " – skipping.")
      next
    }
    
    # Row names for the pre and post respiratory betas
    rn_pre  <- paste0("pre_",  pre_var)
    rn_post <- paste0("post_", post_var)
    
    if (!(rn_pre %in% rownames(coef_tab)) || !(rn_post %in% rownames(coef_tab))) {
      message("Missing beta rows for ", out, " × ", resp, " – skipping.")
      next
    }
    
    # Extract betas and p-values
    beta_pre   <- coef_tab[rn_pre,  "Estimate"]
    beta_post  <- coef_tab[rn_post, "Estimate"]
    p_pre      <- coef_tab[rn_pre,  p_col]
    p_post     <- coef_tab[rn_post, p_col]
    
    # --------------------------
    # Wald test: β_pre = β_post
    # --------------------------
    lh <- tryCatch(
      car::linearHypothesis(
        fits,
        paste0(rn_pre, " - ", rn_post, " = 0")
      ),
      error = function(e) {
        message("linearHypothesis failed for ", out, " × ", resp, ": ", e$message)
        return(NULL)
      }
    )
    
    if (is.null(lh)) next
    if (!all(c("Chisq", p_col) %in% colnames(lh))) {
      message("Invalid test table for ", out, " × ", resp, " – skipping.")
      next
    }
    
    chi2 <- lh[2, "Chisq"]
    p_two <- lh[2, p_col]
    
    if (!is.finite(chi2) || !is.finite(p_two)) {
      message("Skipping ", out, " × ", resp, " because test statistic or p-value were invalid.")
      next
    }
    
    # One-sided p (post > pre)
    p_one <- p_two / 2
    stronger_side <- ifelse(beta_post > beta_pre, "post", "pre")
    
    sur_results[[length(sur_results) + 1]] <- data.frame(
      outcome       = out,
      resp_type     = resp,
      N             = nrow(d),
      beta_post     = beta_post,
      p_post        = p_post,
      beta_pre      = beta_pre,
      p_pre         = p_pre,
      beta_diff     = beta_post - beta_pre,
      chi2          = chi2,
      p_two_tailed  = p_two,
      p_one_tailed  = p_one,
      stronger_side = stronger_side,
      stringsAsFactors = FALSE
    )
  }
}

sur_df <- bind_rows(sur_results)

# ------------------------------------------------------------
# 4. Save to CSV
# ------------------------------------------------------------
write.csv(sur_df, "SUR_beta_comparison_results_with_pvals.csv", row.names = FALSE)
cat("\nSaved: SUR_beta_comparison_results_with_pvals.csv\n")

sur_df



Invalid test table for fss_sum × mip – skipping.
Invalid test table for fss_sum × smip – skipping.
Invalid test table for fss_sum × fit – skipping.
Invalid test table for fss_sum × id – skipping.
Invalid test table for fss_sum × slopesmip – skipping.
Invalid test table for fss_sum × sindex – skipping.
Invalid test table for fss_sum × pif – skipping.
Invalid test table for fss_sum × volume – skipping.
Invalid test table for woods_sum × mip – skipping.
Invalid test table for woods_sum × smip – skipping.
Invalid test table for woods_sum × fit – skipping.
Invalid test table for woods_sum × id – skipping.
Invalid test table for woods_sum × slopesmip – skipping.
Invalid test table for woods_sum × sindex – skipping.
Invalid test table for woods_sum × pif – skipping.
Invalid test table for woods_sum × volume – skipping.
Invalid test table for dsq_sum × mip – skipping.
Invalid test table for dsq_sum × smip – skipping.
Invalid test table for dsq_sum × fit – skipping.
Invalid test table for dsq_s

## OG

In [44]:
library(systemfit)
library(dplyr)

# ------------------------------------------------------------
# 3. LOOP: Run SUR for each outcome × respiratory predictor
# ------------------------------------------------------------
sur_results <- list()

for (out in outcomes) {
  for (resp in names(resp_pairs)) {
    
    pre_var  <- resp_pairs[[resp]][1]
    post_var <- resp_pairs[[resp]][2]
    
    # Skip if columns are missing
    if (!all(c(out, pre_var, post_var, covars) %in% names(df_t1))) next
    
    # Subset and drop rows with missing values
    d <- df_t1 %>%
      dplyr::select(all_of(c(out, pre_var, post_var, covars))) %>%
      na.omit()
    
    # Very small sample = unstable
    if (nrow(d) < 12) next  
    
    # Build formulas
    f_pre  <- as.formula(paste(out, "~", pre_var,  "+", paste(covars, collapse = "+")))
    f_post <- as.formula(paste(out, "~", post_var, "+", paste(covars, collapse = "+")))
    
    # Fit SUR (wrap in tryCatch)
    fits <- tryCatch(
      systemfit(
        list(pre = f_pre, post = f_post),
        method = "SUR",
        data = d
      ),
      error = function(e) {
        message("Skipping ", out, " × ", resp, " due to systemfit error: ", e$message)
        return(NULL)
      }
    )
    
    if (is.null(fits)) next
    
    # Names for the two coefficients we care about
    coef_name_pre  <- paste0("pre_",  pre_var)
    coef_name_post <- paste0("post_", post_var)
    
    coefs <- coef(fits)
    vc    <- vcov(fits)
    
    # Make sure both coefficients exist
    if (!(coef_name_pre %in% names(coefs)) ||
        !(coef_name_post %in% names(coefs))) {
      message("Skipping ", out, " × ", resp, 
              " because one or both betas were not estimated.")
      next
    }
    
    beta_pre  <- as.numeric(coefs[coef_name_pre])
    beta_post <- as.numeric(coefs[coef_name_post])
    beta_diff <- beta_post - beta_pre
    
    # Indices in covariance matrix
    idx_pre  <- which(names(coefs) == coef_name_pre)
    idx_post <- which(names(coefs) == coef_name_post)
    
    var_diff <- vc[idx_post, idx_post] + vc[idx_pre, idx_pre] - 2 * vc[idx_post, idx_pre]
    
    # Guard against weird covariance results
    if (is.na(var_diff) || var_diff <= 0) {
      message("Skipping ", out, " × ", resp,
              " because var_diff was invalid (", var_diff, ").")
      next
    }
    
    se_diff <- sqrt(var_diff)
    z       <- beta_diff / se_diff
    p_two   <- 2 * pnorm(abs(z), lower.tail = FALSE)
    
    # One-sided p for the hypothesis: post beta > pre beta
    p_one_dir <- if (z > 0) {
      pnorm(z, lower.tail = FALSE)  # small if post > pre
    } else {
      1 - pnorm(z, lower.tail = FALSE)  # large if goes opposite direction
    }
    
    sur_results[[length(sur_results) + 1]] <- data.frame(
      outcome       = out,
      resp_type     = resp,
      N             = nrow(d),
      beta_pre      = beta_pre,
      beta_post     = beta_post,
      beta_diff     = beta_diff,
      se_diff       = se_diff,
      z_diff        = z,
      p_two_tailed  = p_two,
      p_one_tailed  = p_one_dir,
      stronger_side = ifelse(beta_post > beta_pre, "post", "pre"),
      stringsAsFactors = FALSE
    )
  }
}

sur_df <- bind_rows(sur_results)

# ------------------------------------------------------------
# 4. Save to CSV
# ------------------------------------------------------------
write.csv(sur_df, "SUR_beta_comparison_results.csv", row.names = FALSE)
cat("\nSaved: SUR_beta_comparison_results.csv\n")
print(sur_df)



Saved: SUR_beta_comparison_results.csv
     outcome resp_type  N      beta_pre     beta_post     beta_diff     se_diff       z_diff
1    fss_sum       mip 22 -0.0564391951 -0.0639640551 -7.524860e-03 0.065335626 -0.115172387
2    fss_sum      smip 22 -0.0030046166 -0.0047290340 -1.724417e-03 0.002945997 -0.585342476
3    fss_sum       fit 22 -0.0154007271 -0.0222563775 -6.855650e-03 0.015246131 -0.449664914
4    fss_sum        id 16 -0.0140366138 -0.0186314085 -4.594795e-03 0.049304440 -0.093192311
5    fss_sum slopesmip 16 -0.0094228818 -0.0066965394  2.726342e-03 0.020377638  0.133790893
6    fss_sum    sindex 22 -0.0963853896 -0.1079559386 -1.157055e-02 0.069368981 -0.166797161
7    fss_sum       pif 22 -1.4172690397 -1.6447929020 -2.275239e-01 1.120519410 -0.203052139
8    fss_sum    volume 22  0.1641424612  0.1724103822  8.267921e-03 0.588764679  0.014042828
9  woods_sum       mip 22 -0.0079994760 -0.0085278802 -5.284042e-04 0.025381571 -0.020818421
10 woods_sum      smip 22  0.0

# OLS Fixed Effects

In [5]:
library(dplyr)
library(ggplot2)
library(broom)

# --------------------------------------------------
# CONFIGURATION
# --------------------------------------------------
outcomes    <- c("fss_sum", "woods_sum", "dsq_sum")
resp_types  <- c("mip", "smip", "fit", "id", "slopesmip", "sindex", "pif", "volume")
#covariates  <- c("data_age", "subject_female", "data_kilograms", "data_centimeters")
covariates <- c()
condition_var <- "condition"   # 0 = pre-treadmill, 1 = post-treadmill

# df_prepost is assumed to already exist from your earlier step
# Structure: 2 rows per participant (condition 0/1),
# columns like: mip, mip_c, smip, smip_c, ..., outcomes, covariates

# --------------------------------------------------
# STORAGE FOR RESULTS
# --------------------------------------------------
reg_results <- list()
reg_idx     <- 1

# --------------------------------------------------
# PDF FOR DIAGNOSTIC PLOTS
# --------------------------------------------------
pdf("fatigue_lm_diagnosticcheck.pdf", width = 8, height = 10)

for (y in outcomes) {
  for (resp in resp_types) {
    
    resp_raw <- resp
    resp_c   <- paste0(resp, "_c")
    
    # Skip if the centered or raw variable doesn't exist
    if (!(resp_raw %in% names(df_prepost)) || !(resp_c %in% names(df_prepost))) {
      warning("Skipping ", resp, " for outcome ", y, ": variables not found in df_prepost")
      next
    }
    
    # Build dataset for this model
    df_model <- df_prepost %>%
      dplyr::select(
        record_id,
        all_of(condition_var),
        all_of(y),
        all_of(covariates),
        all_of(resp_raw),
        all_of(resp_c)
      ) %>%
      na.omit()
    
    # Basic guards
    if (nrow(df_model) < 10) next
    if (length(unique(df_model[[condition_var]])) < 2) next
    if (length(unique(df_model$record_id)) < 5) next
    
    # --------------------------------------------------
    # Build formula: outcome ~ resp_c * condition + covariates
    # --------------------------------------------------
    rhs_terms <- c(
      paste0(resp_c, " * ", condition_var),
      covariates
    )
    formula_str <- paste(y, "~", paste(rhs_terms, collapse = " + "))
    form_lm     <- as.formula(formula_str)
    
    # --------------------------------------------------
    # Fit linear model
    # --------------------------------------------------
    fit <- lm(form_lm, data = df_model)
    
    # --------------------------------------------------
    # Diagnostics: Residuals vs Fitted, QQ, Scale-Location
    # --------------------------------------------------
    par(mfrow = c(3, 1))
    
    plot(
      fit,
      which = 1,
      main = paste("Residuals vs Fitted:", y, "~", resp, "* condition")
    )
    
    plot(
      fit,
      which = 2,
      main = paste("Normal Q-Q:", y, "~", resp, "* condition")
    )
    
    plot(
      fit,
      which = 3,
      main = paste("Scale-Location:", y, "~", resp, "* condition")
    )
    
    # --------------------------------------------------
    # Extract interaction term and model info
    # --------------------------------------------------
    tidy_fit <- broom::tidy(fit)
    glance   <- broom::glance(fit)
    
    term_name <- paste0(resp_c, ":", condition_var)
    row_int   <- tidy_fit[tidy_fit$term == term_name, ]
    
    if (nrow(row_int) == 1) {
      est <- row_int$estimate
      se  <- row_int$std.error
      t   <- row_int$statistic
      p   <- row_int$p.value
    } else {
      est <- se <- t <- p <- NA_real_
    }
    
    reg_results[[reg_idx]] <- data.frame(
      outcome          = y,
      resp_type        = resp,
      predictor        = resp_c,
      N                = nrow(df_model),
      n_subjects       = length(unique(df_model$record_id)),
      beta_interaction = est,
      se_interaction   = se,
      t_interaction    = t,
      p_interaction    = p,
      r_squared        = glance$r.squared,
      adj_r_squared    = glance$adj.r.squared,
      covariates       = if (length(covariates) > 0) paste(covariates, collapse = ", ") else "none",
      stringsAsFactors = FALSE
    )
    reg_idx <- reg_idx + 1
  }
}

dev.off()  # closes fatigue_lm_diagnosticcheck.pdf

# --------------------------------------------------
# COMBINE & SAVE REGRESSION TABLE
# --------------------------------------------------
fatigue_lm_regressionresults <- bind_rows(reg_results)

write.csv(
  fatigue_lm_regressionresults,
  "fatigue_lm_regressionresults.csv",
  row.names = FALSE
)

cat("\n✅ DONE (LM interaction models)!\n")
cat("• Interaction results saved to: fatigue_lm_regressionresults.csv\n")
cat("• Diagnostics PDF saved as: fatigue_lm_diagnosticcheck.pdf\n")


# --------------------------------------------------
# SCATTERPLOTS WITH REGRESSION LINES
# FOR SIGNIFICANT INTERACTIONS (p < .05)
# --------------------------------------------------
sig_threshold <- 0.05

sig_rows <- fatigue_lm_regressionresults %>%
  filter(!is.na(p_interaction) & p_interaction < sig_threshold)

pdf("fatigue_lm_scatterplot.pdf", width = 8, height = 6)

if (nrow(sig_rows) > 0) {
  for (i in seq_len(nrow(sig_rows))) {
    y        <- sig_rows$outcome[i]
    resp     <- sig_rows$resp_type[i]
    resp_raw <- resp  # x-axis for plotting
    
    # Build plot dataset
    df_plot <- df_prepost %>%
      dplyr::select(
        record_id,
        all_of(condition_var),
        all_of(y),
        all_of(resp_raw)
      ) %>%
      na.omit()
    
    p_scatter <- ggplot(
      df_plot,
      aes(x = .data[[resp_raw]],
          y = .data[[y]],
          color = factor(.data[[condition_var]]))
    ) +
      geom_point() +
      geom_smooth(method = "lm", se = FALSE) +
      labs(
        title = paste0(
          "Scatterplot with Regression Lines by Condition\n",
          y, " vs ", resp, " (significant interaction)"
        ),
        x     = resp_raw,
        y     = y,
        color = "Condition\n(0 = pre, 1 = post)"
      )
    
    print(p_scatter)
  }
} else {
  # If no significant interactions, create a placeholder page
  plot.new()
  title("No significant interactions (p < 0.05)\nNo scatterplots generated.")
}

dev.off()  # closes fatigue_lm_scatterplot.pdf

cat("• Scatterplots PDF saved as: fatigue_lm_scatterplot.pdf\n")



✅ DONE (LM interaction models)!
• Interaction results saved to: fatigue_lm_regressionresults.csv
• Diagnostics PDF saved as: fatigue_lm_diagnosticcheck.pdf
• Scatterplots PDF saved as: fatigue_lm_scatterplot.pdf


# OLS Mixed Effects

In [6]:
library(dplyr)
library(ggplot2)
library(lme4)
library(lmerTest)   # for p-values in lmer
library(broom.mixed)

# --------------------------------------------------
# CONFIGURATION
# --------------------------------------------------
outcomes      <- c("fss_sum", "woods_sum", "dsq_sum")
resp_types    <- c("mip", "smip", "fit", "id", "slopesmip", "sindex", "pif", "volume")
covariates    <- c("data_age", "subject_female")
condition_var <- "condition"   # 0 = pre-treadmill, 1 = post-treadmill

# Assumes df_prepost already exists and has:
# record_id, condition, outcomes, covariates,
# resp vars like mip, smip, ..., and centered versions mip_c, smip_c, etc.

# --------------------------------------------------
# STORAGE FOR RESULTS
# --------------------------------------------------
reg_results <- list()
reg_idx     <- 1

# --------------------------------------------------
# PDF FOR DIAGNOSTIC PLOTS
# --------------------------------------------------
pdf("fatigue_lmer_diagnosticcheck.pdf", width = 8, height = 10)

for (y in outcomes) {
  for (resp in resp_types) {
    
    resp_raw <- resp
    resp_c   <- paste0(resp, "_c")
    
    # Check that variables exist
    if (!(resp_raw %in% names(df_prepost)) || !(resp_c %in% names(df_prepost))) {
      warning("Skipping ", resp, " for outcome ", y, ": variables not found in df_prepost")
      next
    }
    
    # Build dataset for this model
    df_model <- df_prepost %>%
      dplyr::select(
        record_id,
        all_of(condition_var),
        all_of(y),
        all_of(covariates),
        all_of(resp_raw),
        all_of(resp_c)
      ) %>%
      na.omit()
    
    # Guards
    if (nrow(df_model) < 10) next
    if (length(unique(df_model[[condition_var]])) < 2) next
    if (length(unique(df_model$record_id)) < 5) next
    
    # --------------------------------------------------
    # Build formula: outcome ~ resp_c * condition + covariates + (1 | record_id)
    # --------------------------------------------------
    rhs_terms <- c(
      paste0(resp_c, " * ", condition_var),
      covariates,
      "(1 | record_id)"
    )
    formula_str <- paste(y, "~", paste(rhs_terms, collapse = " + "))
    form_lmer   <- as.formula(formula_str)
    
    # --------------------------------------------------
    # Fit mixed-effects model (random intercept)
    # --------------------------------------------------
    fit <- tryCatch(
      lmer(form_lmer, data = df_model, REML = TRUE),
      error = function(e) {
        warning("Model failed for outcome ", y, " and resp ", resp, ": ", e$message)
        return(NULL)
      }
    )
    
    if (is.null(fit)) next
    
    # --------------------------------------------------
    # Diagnostics: Residuals vs Fitted, QQ, Scale-Location
    # --------------------------------------------------
    par(mfrow = c(3, 1))
    
    fitted_vals <- fitted(fit)
    resid_vals  <- resid(fit)
    
    # 1) Residuals vs Fitted
    plot(
      fitted_vals,
      resid_vals,
      xlab = "Fitted values",
      ylab = "Residuals",
      main = paste("Residuals vs Fitted (LMM):", y, "~", resp, "* condition")
    )
    abline(h = 0, lty = 2)
    
    # 2) Normal Q-Q of residuals
    qqnorm(
      resid_vals,
      main = paste("Normal Q-Q (LMM residuals):", y, "~", resp, "* condition")
    )
    qqline(resid_vals)
    
    # 3) Scale-Location: sqrt(|resid|) vs fitted
    plot(
      fitted_vals,
      sqrt(abs(resid_vals)),
      xlab = "Fitted values",
      ylab = "sqrt(|Residuals|)",
      main = paste("Scale-Location (LMM):", y, "~", resp, "* condition")
    )
    
    # --------------------------------------------------
    # Extract interaction term and model info
    # --------------------------------------------------
    tidy_fit <- broom.mixed::tidy(fit, effects = "fixed")
    
    term_name <- paste0(resp_c, ":", condition_var)
    row_int   <- tidy_fit[tidy_fit$term == term_name, ]
    
    if (nrow(row_int) == 1) {
      est <- row_int$estimate
      se  <- row_int$std.error
      t   <- row_int$statistic
      p   <- row_int$p.value
    } else {
      est <- se <- t <- p <- NA_real_
    }
    
    # Optional: R² (marginal/conditional) if MuMIn is installed
    marg_r2 <- cond_r2 <- NA_real_
    if (requireNamespace("MuMIn", quietly = TRUE)) {
      r2_vals <- MuMIn::r.squaredGLMM(fit)
      marg_r2 <- r2_vals[1, "R2m"]
      cond_r2 <- r2_vals[1, "R2c"]
    }
    
    reg_results[[reg_idx]] <- data.frame(
      outcome          = y,
      resp_type        = resp,
      predictor        = resp_c,
      N                = nrow(df_model),
      n_subjects       = length(unique(df_model$record_id)),
      beta_interaction = est,
      se_interaction   = se,
      t_interaction    = t,
      p_interaction    = p,
      R2_marginal      = marg_r2,
      R2_conditional   = cond_r2,
      covariates       = if (length(covariates) > 0) paste(covariates, collapse = ", ") else "none",
      stringsAsFactors = FALSE
    )
    reg_idx <- reg_idx + 1
  }
}

dev.off()  # closes fatigue_lmer_diagnosticcheck.pdf

# --------------------------------------------------
# COMBINE & SAVE REGRESSION TABLE
# --------------------------------------------------
fatigue_lmer_regressionresults <- bind_rows(reg_results)

write.csv(
  fatigue_lmer_regressionresults,
  "fatigue_lmer_regressionresults.csv",
  row.names = FALSE
)

cat("\n✅ DONE (LMM interaction models)!\n")
cat("• Interaction results saved to: fatigue_lmer_regressionresults.csv\n")
cat("• Diagnostics PDF saved as: fatigue_lmer_diagnosticcheck.pdf\n")


# --------------------------------------------------
# SCATTERPLOTS WITH REGRESSION LINES
# FOR SIGNIFICANT INTERACTIONS (p < .05)
# --------------------------------------------------
sig_threshold <- 0.05

sig_rows <- fatigue_lmer_regressionresults %>%
  filter(!is.na(p_interaction) & p_interaction < sig_threshold)

pdf("fatigue_lmer_scatterplot.pdf", width = 8, height = 6)

if (nrow(sig_rows) > 0) {
  for (i in seq_len(nrow(sig_rows))) {
    y        <- sig_rows$outcome[i]
    resp     <- sig_rows$resp_type[i]
    resp_raw <- resp
    
    df_plot <- df_prepost %>%
      dplyr::select(
        record_id,
        all_of(condition_var),
        all_of(y),
        all_of(resp_raw)
      ) %>%
      na.omit()
    
    p_scatter <- ggplot(
      df_plot,
      aes(x = .data[[resp_raw]],
          y = .data[[y]],
          color = factor(.data[[condition_var]]))
    ) +
      geom_point() +
      geom_smooth(method = "lm", se = FALSE) +
      labs(
        title = paste0(
          "Scatterplot with Regression Lines by Condition (LMM sig interaction)\n",
          y, " vs ", resp
        ),
        x     = resp_raw,
        y     = y,
        color = "Condition\n(0 = pre, 1 = post)"
      )
    
    print(p_scatter)
  }
} else {
  plot.new()
  title("No significant interactions (p < 0.05)\nNo scatterplots generated (LMM).")
}

dev.off()  # closes fatigue_lmer_scatterplot.pdf

cat("• Scatterplots PDF saved as: fatigue_lmer_scatterplot.pdf\n")



Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step



Warning message:
package ‘lmerTest’ was built under R version 4.3.3 
Warning message:
package ‘broom.mixed’ was built under R version 4.3.3 
There were 50 or more warnings (use warnings() to see the first 50)



✅ DONE (LMM interaction models)!
• Interaction results saved to: fatigue_lmer_regressionresults.csv
• Diagnostics PDF saved as: fatigue_lmer_diagnosticcheck.pdf
• Scatterplots PDF saved as: fatigue_lmer_scatterplot.pdf


# Scatterplots

In [12]:
library(ggplot2)
library(dplyr)

outcomes   <- c("fss_sum", "dsq_sum")
resp_core  <- c("mip", "sindex", "pif")  # what you want to plot

pdf("scatter_resp_vs_outcome_by_condition.pdf", width = 7, height = 5)

for (y in outcomes) {
  for (resp in resp_core) {
    
    x_var <- paste0(resp, "_c")  # centered
    needed <- c("record_id", "condition", y, x_var)
    
    if (!all(needed %in% names(df_prepost))) next
    
    df_plot <- df_prepost %>%
      dplyr::select(all_of(needed)) %>%
      na.omit()
    
    if (nrow(df_plot) < 10) next
    
    p <- ggplot(
      df_plot,
      aes(x = .data[[x_var]], y = .data[[y]], color = factor(condition))
    ) +
      geom_point() +
      geom_smooth(method = "lm", se = FALSE) +
      labs(
        title = paste(y, "vs", resp, "by condition"),
        x     = paste0(resp, " (centered)"),
        y     = y,
        color = "Condition\n0 = pre, 1 = post"
      ) +
      theme_minimal()
    
    print(p)
  }
}

dev.off()

cat("✅ Scatterplots saved to scatter_resp_vs_outcome_by_condition.pdf\n")


`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
✅ Scatterplots saved to scatter_resp_vs_outcome_by_condition.pdf
